In [1]:

#!python3 -m pip install pytorch_lightning

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from model import LSTM_Model
from pytorch_lightning import seed_everything, LightningModule, Trainer
from torch import save
from pytorch_lightning.callbacks import EarlyStopping

torch.manual_seed(1)

Dataset version: 0.15


In [ ]:
seed_everything(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Check for cuda 
device = 'cpu'
print(f'Using {device} device')

early_stop_callback = EarlyStopping(monitor='val_loss', min_delta=0.00, patience=5, verbose=True, mode='min')

model = LSTM_Model('data').to(device)
#trainer = Trainer(max_epochs=100, min_epochs=1, auto_lr_find=False, auto_scale_batch_size=False, callbacks=[early_stop_callback],enable_checkpointing=False)
trainer = Trainer(max_epochs=100, min_epochs=1, auto_lr_find=False, auto_scale_batch_size=False,enable_checkpointing=False)
trainer.tune(model)



trainer.fit(model)
save(model.state_dict(), '/trained_model')

Global seed set to 42


Using cpu device


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/aron/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1764: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name     | Type             | Params
----------------------------------------------
0 | lstm     | LSTM             | 42.0 K
1 | fc       | Linear           | 5.5 M 
2 | flatten  | Flatten          | 0     
3 | output   | Sigmoid          | 0     
4 | loss     | CrossEntropyLoss | 0     
5 | accuracy | Accuracy         | 0     
----------------------------------------------
5.6 M     Trainable params
0         Non-trainable params
5.6 M     Total params
22.311    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/aron/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/home/aron/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:225: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

{'progress_bar': {'val_loss': 84.51029205322266}, 'log': {'val_loss': 84.51029205322266}, 'val_loss': 84.51029205322266}
Accuracy: 0.25793686509132385


Validation: 0it [00:00, ?it/s]

{'progress_bar': {'val_loss': 84.52635955810547}, 'log': {'val_loss': 84.52635955810547}, 'val_loss': 84.52635955810547}
Accuracy: 0.2597940266132355


Validation: 0it [00:00, ?it/s]

{'progress_bar': {'val_loss': 84.51219177246094}, 'log': {'val_loss': 84.51219177246094}, 'val_loss': 84.51219177246094}
Accuracy: 0.26091355085372925
